In [1]:
import pandas as pd
import h2o
import nltk
import numpy as np
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 16.0.1+9-24, mixed mode, sharing)
  Starting server from C:\Users\siraj\AppData\Roaming\Python\Python39\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\siraj\AppData\Local\Temp\tmpojer_v_l
  JVM stdout: C:\Users\siraj\AppData\Local\Temp\tmpojer_v_l\h2o_siraj_started_from_python.out
  JVM stderr: C:\Users\siraj\AppData\Local\Temp\tmpojer_v_l\h2o_siraj_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Rome
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_siraj_of2smk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.947 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
dataset = pd.read_csv("New_Smart.csv")

In [3]:
dataset

,Person,Located_At_TimeStamp,Class
0,"[{""name"":Imran Jami}",{Employee_At},1
1,"[{""name"":Shaukat Wasi}",{Employee_At},1
2,"[{""name"":Zubair Ahmed Shiekh}",{Employee_At},1
3,"[{""name"":Asim Wagan}",{Employee_At},1
4,"[{""name"":Tafseer Ahmed}",{Employee_At},1
...,...,...,...
2126,"[{""name"":Shaukat Wasi}","{""Timestamp"":24/09/2022 5:47}",22
2127,"[{""name"":Shaukat Wasi}","{""Timestamp"":01/08/2022 10:03}",22
2128,"[{""name"":Shaukat Wasi}","{""Timestamp"":16/02/2022 23:26}",22
2129,"[{""name"":Imran Jami}",{Employee_At},16


In [30]:
Facts = dataset.Facts
Reasoning = dataset.Reasoning
Classification = dataset.Class

In [31]:
h2o_df=h2o.H2OFrame(np.hstack((Facts[:,None],Reasoning[:,None],Classification[:,None])),column_names=['Facts','Reasoning','Class'],column_types=['string','string','string'])

Parse progress: |

/tmp/ipykernel_639/1618786265.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  h2o_df=h2o.H2OFrame(np.hstack((Facts[:,None],Reasoning[:,None],Classification[:,None])),column_names=['Facts','Reasoning','Class'],column_types=['string','string','string'])


████████████████████████████████████████████████████████████████| (done) 100%


In [32]:
h2o_df

Facts,Reasoning,Class
"contain(east_africa,kenya).",East Africa contains Kenya,City
"borders(black_sea,romania).",Black sea has border with romania,Location
"borders(united_states,canada).",united states has border with canada,Location
"city(bangkok,thailand,1178).",thailand is a city of bangkok,City
"borders(tonga,pacific).",tonga has border with pacific,Location
"borders(morocco,mediterranean).",morocco has border with mediteranean,Location
"contain(mozambique,limpopo).",mozambique contains limpopo,City
"borders(indian_ocean,sri_lanka).",indian ocean has border with srilanka,Location
"borders(burundi,tanzania).",burundi has border with tanzania,Location
"contain(singapore,singapore_city).",singapore has singapore city,City


In [33]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [34]:
def tokenize(line):
    tokenized = line.tokenize("\\W+")
    tokenized = tokenized.tolower()
    tokenized = tokenized[(tokenized.nchar() >= 2) | (tokenized.isna()),:]
    tokenized_filter = tokenized[(tokenized.isna()) | (~ tokenized.isin(stop_words)),:]
    return tokenized_filter

In [35]:
final_data = tokenize(h2o_df["Facts"])
final_data_2 = tokenize(h2o_df["Reasoning"])

In [36]:
final_data.head()

C1
contain
east_africa
kenya
""
borders
black_sea
romania
""
borders
united_states


In [37]:
from h2o.estimators.word2vec import H2OWord2vecEstimator

In [38]:
vec_model = H2OWord2vecEstimator(vec_size = 100, model_id = "Vector_Model.model")
vec_model.train(training_frame=final_data)
vec_model.train(training_frame=final_data_2)

word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%
word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OWord2vecEstimator : Word2Vec
Model Key: Vector_Model.model

No summary for this model

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [39]:
h2o.save_model(vec_model,path='./')

'/notebook/Vector_Model.model'

In [46]:
data_vecs_facts = vec_model.transform(final_data, aggregate_method = "AVERAGE")

In [47]:
data_vecs_facts.shape

(1982, 100)

In [48]:
data_vecs_facts.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
-0.0804315,0.0261309,-0.0774636,0.068279,-0.0496401,-0.0286812,-0.0218975,-0.0894714,-0.0665537,0.128444,0.0930677,0.00603164,-0.0742414,0.0500423,-0.0381064,0.0672356,0.0289532,0.0834457,0.0367387,0.0275211,-0.0389412,0.105757,0.031798,0.027442,-0.0567548,0.0126397,-0.041249,-0.0334738,0.0589358,-0.047043,-0.0479197,-0.0347731,0.00677878,0.00640974,0.0447724,-0.0169004,-0.0523166,-0.0822592,0.0775725,0.0349141,-0.0328983,0.0127107,-0.0155929,-0.0648609,-0.0678146,-0.0720191,0.0272035,0.0624181,-0.058371,-0.0362557,-0.0260832,-0.0354629,0.0342102,0.00282877,0.0414538,-0.0431819,0.0492525,0.0757163,0.00872406,0.0606934,0.04359,0.0800438,-0.0693476,0.0178327,-0.0406145,-0.0610246,-0.0950007,0.119919,0.0265634,-0.0258238,0.0180989,0.030659,0.0306272,-0.00739072,-0.0519539,0.00177069,-0.0172618,0.0507531,0.0277696,-0.0105335,0.0525625,0.00415551,-0.0193126,-0.031511,0.0505257,-0.0105934,-0.0445901,-0.00278276,-0.0519399,-0.0505231,0.0404999,0.0827795,0.0428495,0.0388431,-0.0142542,0.0119188,0.00556841,0.0203015,-0.0221777,-0.0378009
-0.0388504,0.0207406,-0.0536098,0.0812762,-0.0406263,-0.0419067,-0.0308263,-0.0387313,-0.0603023,0.075504,0.0937592,-0.0152044,-0.0513502,-0.00167487,-0.00280645,0.0485988,0.00464617,0.0702378,0.0385195,0.00694385,-0.042448,0.0601769,0.0425082,-0.00169651,-0.0345114,0.0133852,-0.0370356,-0.0337528,0.0115897,-0.0312547,-0.0280953,-0.0131321,0.0147554,0.0165221,0.0477311,-0.0294728,-0.0443455,-0.0431409,0.0249611,0.0401016,-0.0134427,0.00280008,-0.00563951,-0.0447456,-0.0541336,-0.0494705,0.0603105,0.0631035,-0.0684815,-0.0148819,-0.0114694,-0.0105462,0.0426259,0.00451178,0.00872244,-0.0134557,0.00493163,0.0399771,0.00910776,0.0765661,0.015773,0.0720286,-0.0540389,0.011504,-0.00362033,-0.0475564,-0.0695076,0.074343,0.0363933,-0.0358417,0.0226589,0.0364282,0.0224252,0.0138146,-0.0507312,0.0123863,0.00263454,0.0279962,0.0375715,-0.0121384,0.0803639,0.0329165,-0.0143166,-0.0416585,0.00692202,-0.0297107,-0.0626901,-0.00948152,0.00411711,-0.0147413,0.0582894,0.0548645,0.0327838,0.0171448,0.0120651,0.0356289,-0.0288535,0.0182726,-0.0175184,-0.0240013
-0.0531694,0.00154472,-0.0559632,0.0796309,-0.0808246,-0.0567622,-0.0316918,-0.074014,-0.0626894,0.14892,0.110597,-0.00457543,-0.0664752,0.0112351,-0.00887626,0.0684974,0.0525809,0.0829079,0.0405803,0.0295407,-0.0378097,0.065327,0.0619523,-0.00338832,-0.0449561,0.00560187,-0.0269133,-0.069329,0.0380181,-0.0827964,-0.0352292,-0.00159639,-0.0254669,0.0152545,0.033346,-0.0258275,-0.0683097,-0.0428693,0.0579692,0.0266867,-0.0273172,0.0180297,-0.0115766,-0.0513027,-0.0635705,-0.0720138,0.0753464,0.0760136,-0.0839378,-0.0427272,-0.024405,-0.00860908,0.0417563,-0.0195159,0.00864938,-0.0409627,0.0534481,0.0650009,-0.028698,0.0460738,-0.00934237,0.0690281,-0.0880734,0.00433452,-0.0149875,-0.0607226,-0.102111,0.108782,0.0137059,-0.019202,0.0741401,0.0445416,0.0325718,0.00149467,-0.0468479,-0.00521604,0.0455589,0.0629763,0.0414014,-0.00983637,0.0937988,0.021946,-0.0253393,-0.00909544,0.0749796,-0.00524197,-0.0856995,0.0147263,-0.0598081,-0.0405211,0.0606953,0.0621302,0.0410079,0.0317904,0.0206919,-0.0012245,-0.00380589,0.0240807,-0.000272182,-0.00762004
-0.0454748,0.00145013,-0.0394667,0.0736647,-0.0562465,-0.062061,-0.0281855,-0.0506952,-0.0668029,0.120076,0.108071,-0.0148935,-0.0617351,-0.00205707,-0.00688701,0.0607763,0.0343268,0.0780443,0.0385011,0.0236546,-0.0348751,0.0475296,0.059109,-0.00632805,-0.0366209,-0.000251536,-0.032151,-0.0566051,0.0121177,-0.0617131,-0.0280522,0.00682086,-0.0141926,0.013213,0.038709,-0.0222748,-0.0597974,-0.0339165,0.0454617,0.0360407,-0.0234069,0.0164469,-

In [50]:
data_vecs_reasoning = vec_model.transform(final_data_2, aggregate_method = "AVERAGE")

In [51]:
data_vecs_reasoning.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
-0.0955537,0.0227056,-0.0826323,0.0669231,-0.0758495,-0.0291848,-0.0302463,-0.121868,-0.0590102,0.161998,0.0957985,0.0216485,-0.0688495,0.0686469,-0.0496524,0.0769222,0.0529706,0.0919822,0.0342285,0.0481539,-0.0305956,0.121723,0.0302369,0.0278315,-0.06588,0.0039638,-0.0341607,-0.0478634,0.0869457,-0.0715223,-0.0502212,-0.0417265,-0.0148793,0.00274794,0.0363177,-0.0171247,-0.0584425,-0.0829002,0.102989,0.0199451,-0.0421176,0.024258,-0.0174795,-0.0710338,-0.0710551,-0.0827153,0.0311662,0.0644972,-0.062609,-0.0551225,-0.0368391,-0.0368986,0.0266017,-0.0135813,0.0505571,-0.0588327,0.0755823,0.088968,-0.0170639,0.038856,0.0369186,0.0788971,-0.0843476,0.0120742,-0.05323,-0.0600203,-0.113538,0.146997,0.0107255,-0.0122131,0.0398987,0.0238642,0.0294213,-0.024307,-0.0457827,-0.00875508,-0.00272651,0.0681676,0.0226644,-0.00841122,0.0457793,-0.000536633,-0.0247588,-0.011718,0.093044,-0.00127634,-0.0474762,0.0109455,-0.0886114,-0.0717611,0.0315977,0.080154,0.0456371,0.04462,-0.0156425,-0.012875,0.0234127,0.0323815,-0.0119912,-0.032029
-0.0579118,0.0256727,-0.0666183,0.06702,-0.0289355,-0.022798,-0.0225601,-0.055259,-0.050518,0.0557932,0.0740973,-0.00367521,-0.0546481,0.0305947,-0.0265315,0.047944,-0.00544292,0.0618076,0.027762,0.00338339,-0.038175,0.0813954,0.0190314,0.0196629,-0.0409799,0.0089468,-0.0365154,-0.0178897,0.0370971,-0.0114532,-0.0318849,-0.032616,0.0236767,0.00646596,0.0520419,-0.0197612,-0.0301067,-0.0621958,0.0438991,0.0334421,-0.0204398,0.00454733,-0.0106372,-0.04995,-0.0543792,-0.0483324,0.021223,0.0451649,-0.0450833,-0.015662,-0.0110166,-0.0219149,0.0282094,0.0149125,0.0329074,-0.0127461,0.00735124,0.0470049,0.0205697,0.0667071,0.0428297,0.0666269,-0.0415404,0.0188904,-0.0198752,-0.0446972,-0.0681713,0.083358,0.0348051,-0.0298375,-0.00583015,0.0188032,0.0152507,-0.00336868,-0.0421769,0.0126106,-0.0289198,0.0260056,0.024628,-0.00850695,0.0462221,0.0153973,-0.0102465,-0.040203,0.00507288,-0.0356663,-0.0319207,-0.0120626,-0.00353059,-0.0247007,0.0340728,0.0624406,0.0331092,0.0258141,-0.0110359,0.027038,-0.0128056,0.016617,-0.0281644,-0.0353679
-0.069974,0.0164063,-0.0709016,0.0686916,-0.0489714,-0.0258271,-0.0284127,-0.0782003,-0.0512151,0.10415,0.081197,0.00454583,-0.0595978,0.0390367,-0.0307198,0.0575346,0.0204409,0.0731132,0.0241767,0.0188211,-0.0335626,0.0906839,0.0309532,0.0198454,-0.0459636,0.00823602,-0.0280072,-0.038159,0.0592921,-0.0490972,-0.0318513,-0.0277269,0.00210288,0.00800649,0.0421686,-0.024393,-0.0453306,-0.0608111,0.0650434,0.0203239,-0.025405,0.0138675,-0.014742,-0.0556959,-0.0619742,-0.0637806,0.031285,0.0552382,-0.0567673,-0.0358044,-0.0235582,-0.0199502,0.027736,0.00479191,0.0396284,-0.0282134,0.0362013,0.0656796,-0.00194683,0.0491331,0.0264659,0.0654909,-0.063177,0.0159569,-0.0275859,-0.0562534,-0.0851872,0.109243,0.0249688,-0.0188856,0.0223222,0.0239545,0.0212007,-0.013818,-0.041841,0.00298843,-0.0011631,0.0492426,0.0207628,-0.01019,0.0513626,0.00749692,-0.0174331,-0.0250976,0.0502019,-0.0227203,-0.0466227,0.00142912,-0.0422286,-0.0436472,0.0360975,0.0640842,0.0399593,0.0328799,-0.0104532,0.00346169,0.00290511,0.0204944,-0.0162675,-0.0244906
-0.0454748,0.00145013,-0.0394667,0.0736647,-0.0562465,-0.062061,-0.0281855,-0.0506952,-0.0668029,0.120076,0.108071,-0.0148935,-0.0617351,-0.00205707,-0.00688701,0.0607763,0.0343268,0.0780443,0.0385011,0.0236546,-0.0348751,0.0475296,0.059109,-0.00632805,-0.0366209,-0.000251536,-0.032151,-0.0566051,0.0121177,-0.0617131,-0.0280522,0.00682086,-0.0141926,0.013213,0.038709,-0.0222748,-0.0597974,-0.0339165,0.0454617,0.0360407,-0.0234069,0.0164469,-0.0171448,-0.0433031,-0.0539356,-0.06

In [52]:
h2o_df_ext=h2o_df.cbind(data_vecs_facts)
h2o_df_ext = h2o_df_ext.cbind(data_vecs_reasoning)

In [53]:
h2o_df_ext.head()

Facts,Reasoning,Class,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C110,C210,C310,C410,C510,C610,C710,C810,C910,C101,C1100,C120,C130,C140,C150,C160,C170,C180,C190,C200,C2100,C220,C230,C240,C250,C260,C270,C280,C290,C300,C3100,C320,C330,C340,C350,C360,C370,C380,C390,C400,C4100,C420,C430,C440,C450,C460,C470,C480,C490,C500,C5100,C520,C530,C540,C550,C560,C570,C580,C590,C600,C6100,C620,C630,C640,C650,C660,C670,C680,C690,C700,C7100,C720,C730,C740,C750,C760,C770,C780,C790,C800,C8100,C820,C830,C840,C850,C860,C870,C880,C890,C900,C9100,C920,C930,C940,C950,C960,C970
"contain(east_africa,kenya).",East Africa contains Kenya,City,-0.0804315,0.0261309,-0.0774636,0.068279,-0.0496401,-0.0286812,-0.0218975,-0.0894714,-0.0665537,0.128444,0.0930677,0.00603164,-0.0742414,0.0500423,-0.0381064,0.0672356,0.0289532,0.0834457,0.0367387,0.0275211,-0.0389412,0.105757,0.031798,0.027442,-0.0567548,0.0126397,-0.041249,-0.0334738,0.0589358,-0.047043,-0.0479197,-0.0347731,0.00677878,0.00640974,0.0447724,-0.0169004,-0.0523166,-0.0822592,0.0775725,0.0349141,-0.0328983,0.0127107,-0.0155929,-0.0648609,-0.0678146,-0.0720191,0.0272035,0.0624181,-0.058371,-0.0362557,-0.0260832,-0.0354629,0.0342102,0.00282877,0.0414538,-0.0431819,0.0492525,0.0757163,0.00872406,0.0606934,0.04359,0.0800438,-0.0693476,0.0178327,-0.0406145,-0.0610246,-0.0950007,0.119919,0.0265634,-0.0258238,0.0180989,0.030659,0.0306272,-0.00739072,-0.0519539,0.00177069,-0.0172618,0.0507531,0.0277696,-0.0105335,0.0525625,0.00415551,-0.0193126,-0.031511,0.0505257,-0.0105934,-0.0445901,-0.00278276,-0.0519399,-0.0505231,0.0404999,0.0827795,0.0428495,0.0388431,-0.0142542,0.0119188,0.00556841,0.0203015,-0.0221777,-0.0378009,-0.0955537,0.0227056,-0.0826323,0.0669231,-0.0758495,-0.0291848,-0.0302463,-0.121868,-0.0590102,0.161998,0.0957985,0.0216485,-0.0688495,0.0686469,-0.0496524,0.0769222,0.0529706,0.0919822,0.0342285,0.0481539,-0.0305956,0.121723,0.0302369,0.0278315,-0.06588,0.0039638,-0.0341607,-0.0478634,0.0869457,-0.0715223,-0.0502212,-0.0417265,-0.0148793,0.00274794,0.0363177,-0.0171247,-0.0584425,-0.0829002,0.102989,0.0199451,-0.0421176,0.024258,-0.0174795,-0.0710338,-0.0710551,-0.0827153,0.0311662,0.0644972,-0.062609,-0.0551225,-0.0368391,-0.0368986,0.0266017,-0.0135813,0.0505571,-0.0588327,0.0755823,0.088968,-0.0170639,0.038856,0.0369186,0.0788971,-0.0843476,0.0120742,-0.05323,-0.0600203,-0.113538,0.146997,0.0107255,-0.0122131,0.0398987,0.0238642,0.0294213,-0.024307,-0.0457827,-0.00875508,-0.00272651,0.0681676,0.0226644,-0.00841122,0.0457793,-0.000536633,-0.0247588,-0.011718,0.093044,-0.00127634,-0.0474762,0.0109455,-0.0886114,-0.0717611,0.0315977,0.080154,0.0456371,0.04462,-0.0156425,-0.012875,0.0234127
"borders(black_sea,romania).",Black sea has border with romania,Location,-0.0388504,0.0207406,-0.0536098,0.0812762,-0.0406263,-0.0419067,-0.0308263,-0.0387313,-0.0603023,0.075504,0.0937592,-0.0152044,-0.0513502,-0.00167487,-0.00280645,0.0485988,0.00464617,0.0702378,0.0385195,0.00694385,-0.042448,0.0601769,0.0425082,-0.00169651,-0.0345114,0.0133852,-0.0370356,-0.0337528,0.0115897,-0.0312547,-0.0280953,-0.0131321,0.0147554,0.0165221,0.0477311,-0.0294728,-0.0443455,-0.0431409,0.0249611,0.0401016,-0.0134427,0.00280008,-0.00563951,-0.0447456,-0.0541336,-0.0494705,0.0603105,0.0631035,-0.0684815,-0.0148819,-0.0114694,-0.0105462,0.0426259,0.00451178,0.00872244,-0.0134557,0.00493163,0.0399771,0.00910776,0.0765661,0.015773,0.0720286,-0.0540389,0.011504,-0.00362033,-0.0475564,-0.0695076,0.074343,0.0363933,-0.0358417,0.0226589,0.0364282,0.0224252,0.0138146,-0.0507312,0.0123863,0.00263454,0.0279962,0.0375715,-0.0121384,0.0803639,0.0329165,-0.0143166,-0.0416585,0.00692202,-0.02

In [59]:
vectorzied_dataset = h2o.as_list(h2o_df_ext, use_pandas=True)

In [60]:
vectorzied_dataset.to_csv("Vectorized_version.csv")